# Custom Objective Function for XGBoost

Noel's notes:
* Working as expected with the custom objective function
* Has the same output as XGBClassifier on either `predict()` or `predict_proba()`

In [ ]:
from typing import Tuple
from scipy.special import expit as sigmoid


def logistic_obj(labels: np.ndarray, predt: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  '''
  Logistic loss objective function for binary-class classification
  '''
  y = labels
  p = sigmoid(predt)
  grad = p - y
  hess = p * (1.0 - p)

  return grad, hess


# Create XGBClassifier model instance with custom objective function
clf = XGBClassifier(random_state=4, n_estimators=1000, objective=logistic_obj)

# fit model
clf.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              objective=<function logistic_obj at 0x7f28a86e20e0>,
              predictor=None, ...)

In [ ]:
# Create XGBClassifier model instance
base_model = XGBClassifier(random_state=4, n_estimators=1000)

# fit model
base_model.fit(train_X, train_y)

## Simulation Test

In [ ]:
from xgboost import DMatrix

sample = test_X.sample()

print(f'''
Customized XGBClassifier
{clf.predict(sample)}
{clf.predict_proba(sample)}

Base XGBClassifier
{base_model.predict(sample)}
{base_model.predict_proba(sample)}
{base_model.get_booster().predict(DMatrix(sample))} # This is actually the source for predict_proba() method
''')

In [ ]:
# Loop through test_X
hit_for_predict_proba = False
hit_for_predict = False

for i in range(len(test_X)):
  # get sample
  sample = test_X.iloc[i:i + 1]

  # compare predictions
  if clf.predict_proba(sample)[:, 1] != base_model.predict_proba(sample)[:, 1]:
    print(clf.predict_proba(sample))
    print(base_model.predict_proba(sample))
    print()
    hit_for_predict_proba = True
  if clf.predict(sample) != base_model.predict(sample):
    print(clf.predict(sample))
    print(base_model.predict(sample))
    hit_for_predict = True

if not hit_for_predict:
  print('Success! Same output in terms of predict()')
if not hit_for_predict_proba:
  print('Success! Same output in terms of predict_proba()')

# Custom Objective Function

Noel's notes:
* Working as expected with the custom objective function
* But, I want to customize `y_pred` in `custom_objective()`

In [ ]:
# Define a custom objective function
def custom_objective(y_true, y_pred):
  # Customize the objective function calculation here
  # y_true: array-like, shape (n_samples,)
  # y_pred: array-like, shape (n_samples, n_classes)
  # Return a scalar representing the objective function value

  # Example: Negative log-likelihood
  return -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


# Create a LogisticRegression object
custom_obj_model = LogisticRegression(solver='lbfgs')

# Set the scoring function to your custom objective function
custom_obj_model.score = custom_objective

# Fit the model to your data
custom_obj_model.fit(train_X_scaled_normalized, train_y.values.ravel())

## Generate forecasts

In [ ]:
# Make forecasts
forecasts = custom_obj_model.predict(
  test_X_scaled_normalized
)

# Get the probability for 1s class
forecasts_proba = custom_obj_model.predict_proba(
  test_X_scaled_normalized
)[:, 1]

forecasts_output = pd.DataFrame(
  {
    'patient_id': [USER] * len(forecasts),
    'ground_truth': test_y.values.ravel(),
    'forecasted_wearing_off': forecasts,
    'forecasted_wearing_off_probability': forecasts_proba
  },
  columns=['patient_id', 'ground_truth', 'forecasted_wearing_off',
           'forecasted_wearing_off_probability'],
  index=test_X_scaled_normalized.index
)
# forecasts_output

## Evaluation
From this part, we're showing how the forecasts_output will be evaluated for each patient.

In [ ]:
# Plot `test_y.values.ravel()` and `preds_proba` on the same plot to show the difference
plt.figure(figsize=FIGSIZE)
plt.plot(forecasts_output.ground_truth,
         label='actual', color='red', marker='o',)
plt.plot(forecasts_output.forecasted_wearing_off_probability,
         label='predicted', color='blue', marker='o')
# plt.plot(forecasts_output.forecasted_wearing_off,
#          label='predicted', color='blue', marker='o')
plt.legend()

# Dashed horizontal line at 0.5
plt.axhline(0.5, linestyle='--', color='gray')

# Dashed vertical lines on each hour
for i in forecasts_output.index:
  if pd.Timestamp(i).minute == 0:
    plt.axvline(i, linestyle='--', color='gray')

# y-axis label Wearing-off Forecast Probability
plt.ylabel('Wearing-off Forecast Probability')

# title
plt.title(f'Custom Objective Function Model for {USER.upper()}')

plt.show()

In [ ]:
# evaluate predictions with f1 score, precision, recall, and accuracy
import sklearn.metrics as metrics
from sklearn.metrics import classification_report
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

display(
  pd.DataFrame(
    [
      metrics.f1_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off),
      metrics.recall_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off),
      metrics.precision_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off),
      metrics.accuracy_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off)
    ],
    index=['f1 score', 'recall', 'precision', 'accuracy'],
    columns=['metrics']
  ).T
)
pd.DataFrame(classification_report(
  forecasts_output.ground_truth,
  forecasts_output.forecasted_wearing_off,
  output_dict=True)).T

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

LABELS = ['No Wearing-off', 'Wearing-off']

conf_matrix = confusion_matrix(test_y.values.ravel(),
                               forecasts_output.forecasted_wearing_off)
plt.figure(figsize=(FIGSIZE_CM))
sns.heatmap(conf_matrix, xticklabels=LABELS,
            yticklabels=LABELS, annot=True, fmt=".2f")
plt.title(
    f"Custom Objective Function Model's confusion matrix for {USER.upper()}")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

# Custom Model

Noel's notes:
* Working but does not have the same output as original

In [ ]:
import numpy as np
from scipy.optimize import minimize


def sigmoid(z):
  return 1 / (1 + np.exp(-z))


def negative_log_likelihood(theta, X, y, alpha):
  m = X.shape[0]  # number of training examples
  z = np.dot(X, theta)
  h = sigmoid(z)
  cost = -np.sum(y * np.log(h) + (1 - y) * np.log(1 - h)) / m
  # Add L2 penalty to the cost function
  #   Exclude theta[0] from regularization
  regularization_term = (alpha / (2 * m)) * np.sum(theta[1:]**2)
  cost += regularization_term
  return cost


def gradient(theta, X, y, alpha):
  m = X.shape[0]  # number of training examples
  z = np.dot(X, theta)
  h = sigmoid(z)
  grad = np.dot(X.T, (h - y)) / m
  # L2 penalty term
  regularization_term = (alpha / m) * theta[1:]
  grad[1:] += regularization_term
  return grad


class CustomLogisticRegression:
  def __init__(self, alpha=0.0):
    self.theta = None
    self.alpha = alpha

  def fit(self, X, y):
    m, n = X.shape
    self.theta = np.zeros(n)

    result = minimize(
        fun=negative_log_likelihood,
        x0=self.theta,
        args=(X, y, self.alpha),
        method='L-BFGS-B',
        jac=gradient,
        options={'maxiter': 200}
    )

    self.theta = result.x

  def predict(self, X):
    z = np.dot(X, self.theta)
    predictions = sigmoid(z)
    return np.round(predictions)

  def predict_proba(self, X):
    z = np.dot(X, self.theta)
    predictions = sigmoid(z)

    # Return the probability for the 0s and 1s class
    predictions_proba = np.zeros((len(predictions), 2))
    predictions_proba[:, 0] = 1 - predictions
    predictions_proba[:, 1] = predictions

    return predictions_proba


custom_lr_model = CustomLogisticRegression(alpha=0.1)
custom_lr_model.fit(train_X_scaled_normalized, train_y.values.ravel())

## Generate forecasts

In [ ]:
# Make forecasts
forecasts = custom_lr_model.predict(
  test_X_scaled_normalized
)

# Get the probability for 1s class
forecasts_proba = custom_lr_model.predict_proba(
  test_X_scaled_normalized
)[:, 1]

forecasts_output = pd.DataFrame(
  {
    'patient_id': [USER] * len(forecasts),
    'ground_truth': test_y.values.ravel(),
    'forecasted_wearing_off': forecasts,
    'forecasted_wearing_off_probability': forecasts_proba
  },
  columns=['patient_id', 'ground_truth', 'forecasted_wearing_off',
           'forecasted_wearing_off_probability'],
  index=test_X_scaled_normalized.index
)
# forecasts_output

## Evaluation
From this part, we're showing how the forecasts_output will be evaluated for each patient.

In [ ]:
# Plot `test_y.values.ravel()` and `preds_proba` on the same plot to show the difference
plt.figure(figsize=FIGSIZE)
plt.plot(forecasts_output.ground_truth,
         label='actual', color='red', marker='o',)
plt.plot(forecasts_output.forecasted_wearing_off_probability,
         label='predicted', color='blue', marker='o')
# plt.plot(forecasts_output.forecasted_wearing_off,
#          label='predicted', color='blue', marker='o')
plt.legend()

# Dashed horizontal line at 0.5
plt.axhline(0.5, linestyle='--', color='gray')

# Dashed vertical lines on each hour
for i in forecasts_output.index:
  if pd.Timestamp(i).minute == 0:
    plt.axvline(i, linestyle='--', color='gray')

# y-axis label Wearing-off Forecast Probability
plt.ylabel('Wearing-off Forecast Probability')

# title
plt.title(f'Custom LR Model for {USER.upper()}')

plt.show()

In [ ]:
# evaluate predictions with f1 score, precision, recall, and accuracy
import sklearn.metrics as metrics
from sklearn.metrics import classification_report
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

display(
  pd.DataFrame(
    [
      metrics.f1_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off),
      metrics.recall_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off),
      metrics.precision_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off),
      metrics.accuracy_score(
        forecasts_output.ground_truth,
        forecasts_output.forecasted_wearing_off)
    ],
    index=['f1 score', 'recall', 'precision', 'accuracy'],
    columns=['metrics']
  ).T
)
pd.DataFrame(classification_report(
  forecasts_output.ground_truth,
  forecasts_output.forecasted_wearing_off,
  output_dict=True)).T

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

LABELS = ['No Wearing-off', 'Wearing-off']

conf_matrix = confusion_matrix(test_y.values.ravel(),
                               forecasts_output.forecasted_wearing_off)
plt.figure(figsize=(FIGSIZE_CM))
sns.heatmap(conf_matrix, xticklabels=LABELS,
            yticklabels=LABELS, annot=True, fmt=".2f")
plt.title(
    f"Custom LR Model's confusion matrix for {USER.upper()}")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()